## Reduction operation: the sum of the numbers in the range [0, value)

In [7]:
import numpy as np
import sys

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Secuencial

#value = 5*10**4
value = int(sys.argv[1])

X = np.random.rand(value)

# Para imprimir los pimeros valores del array

# print(X[0:12])

# Utilizando las operaciones mágicas de ipython

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("Time taken by reduction operation using a function:", tiempo)


print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")


# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)

print("Now, the result using numpy.sum():", np.sum(X),"\n ")


# Utilizando numpy.ndarray.sum()

tiempo= %timeit -r 2 -o -q X.sum()

print("Time taken by reduction operation using numpy.ndarray.sum():", tiempo)

print("Now, the result using numpy.ndarray.sum():", X.sum())




Time taken by reduction operation using a function: 45.4 ms ± 630 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
And the result of the sum of numbers in the range [0, value) is: 24981.510655074384

Time taken by reduction operation using numpy.sum(): 96.8 µs ± 464 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
Now, the result using numpy.sum(): 24981.51065507439 
 
Time taken by reduction operation using numpy.ndarray.sum(): 74.3 µs ± 89.1 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
Now, the result using numpy.ndarray.sum(): 24981.51065507439


### Apartado 3.2 a)

In [8]:
import numpy as np
import multiprocessing
import time
import sys


def reduc_operation(A):
    """
    Suma los elementos de un array A.
    Recibe por argumento A que será el array
    Devuelve s que computa la suma de los elementos del array en el rango [0, value)
    """
    s = 0
    for i in range(A.size):  # Recorremos cada elemento del array
        s += A[i]  # Acumulamos la suma
    return s


def split_array_custom(array, num_parts):
    """
    Dividimos el array en partes iguales segun el numero de procesos que se pide
    """
    size = array.size
    ranges = []

    if num_parts == 1:
        # Con solo un proceso 
        # El array solo es 1 y va desde el inicio[0] hasta su tamaño[array.size] 
        ranges = [(0, size)]
    elif num_parts == 2:
        # Cuando tenemos dos procesos, dividimos en [0, int(size/2)] y [int(size/2), size]
        mid = size // 2
        ranges = [(0, mid), (mid, size)]
    elif num_parts == 4:
        # Si tenemos 4 procesos se dividirá en 1/4 
        q1 = size // 4
        q2 = size // 2
        q3 = 3 * size // 4
        ranges = [(0, q1), (q1, q2), (q2, q3), (q3, size)]
    
    # Usamos los rangos calculados para generar las partes en las que dividimos el array original 
    return [array[start:end] for start, end in ranges]


if __name__ == "__main__":
    # Creamos un array de números aleatorios de tamaño 50*10^7 con np.random.rand
    #value = 5 * 10**7
    value = int(sys.argv[1])
    X = np.random.rand(value)  
    print(f"Tamaño del array: {X.size}\n")

    # 1. Suma secuencial
    print("Suma secuencial usando reduc_operation")
    # medimos el tiempo usando time.time
    start_time = time.time()  
    result = reduc_operation(X)  
    end_time = time.time()  
    print(f"Resultado: {result}")  # Muestro el resultado de la suma
    print(f"Tiempo: {end_time - start_time:.6f} segundos\n")  # Muestro el tiempo
    
    # 2. Paralelismo con multiprocessing
    for num_procs in [1, 2, 4]:  # Aqui iteramos sobre el numero de procesos
        print(f"Paralelismo usando {num_procs} procesos")
        array_parts = split_array_custom(X, num_procs)  # Usamos la funcion anterior para dividir el array en las partes correspondientes
    
        start_time = time.time()  
        # Creamos el pool de procesos
        with multiprocessing.Pool(processes=num_procs) as pool:
            # Con map ejecutar reduc_operation en paralelo para cada parte
            partial_results = pool.map(reduc_operation, array_parts)
            total_sum = sum(partial_results)  # Combinamos las sumas parciales
        end_time = time.time() 
    
        print(f"Resultado: {total_sum}")
        print(f"Tiempo: {end_time - start_time:.6f} segundos\n")

Tamaño del array: 50000000

Suma secuencial usando reduc_operation
Resultado: 24997398.75079772
Tiempo: 46.792098 segundos

Paralelismo usando 1 procesos
Resultado: 24997398.75079772
Tiempo: 49.195225 segundos

Paralelismo usando 2 procesos
Resultado: 24997398.75078987
Tiempo: 26.370568 segundos

Paralelismo usando 4 procesos
Resultado: 24997398.75079415
Tiempo: 13.751142 segundos



### Apartado 3.2 b)

In [9]:
import numpy as np
from numba import njit, prange
import sys

# Usamos njit
@njit
def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in prange(A.size):
        s += A[i]
    return s

# Usamos njit más paralelización
@njit(parallel=True)
def reduc_operation_parallel(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in prange(A.size):
        s += A[i]
    return s




In [10]:


#value = 5*10**7
value = int(sys.argv[1])

X = np.random.rand(value)

tiempo1 = %timeit -r 2 -o -q reduc_operation(X)

print("Time taken by reduction operation using a function with numba:", tiempo1)

tiempo2 = %timeit -r 2 -o -q reduc_operation_parallel(X)

print("Time taken by reduction operation using a function with numba and parallel true:", tiempo2)

Time taken by reduction operation using a function with numba: 200 ms ± 2.76 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Time taken by reduction operation using a function with numba and parallel true: 21.3 ms ± 6.33 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


### Apartado 3.2 d)

Time taken by reduction operation using a function: 4.29 s ± 195 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 25001620.26445533

Time taken by reduction operation using numpy.sum(): 19.1 ms ± 1.9 µs per loop (mean ± std. dev. of 2 runs, 100 loops each)
Now, the result using numpy.sum(): 25001620.26444769 
 
Time taken by reduction operation using numpy.ndarray.sum(): 19.1 ms ± 3.89 µs per loop (mean ± std. dev. of 2 runs, 100 loops each)
Now, the result using numpy.ndarray.sum(): 25001620.26444769
Tamaño del array: 50000000

Suma secuencial usando reduc_operation
Resultado: 25003435.982528258
Tiempo: 4.423266 segundos

Paralelismo usando 1 procesos
Resultado: 25003435.982528258
Tiempo: 4.844689 segundos

Paralelismo usando 2 procesos
Resultado: 25003435.982539788
Tiempo: 2.612920 segundos

Paralelismo usando 4 procesos
Resultado: 25003435.98254129
Tiempo: 1.492263 segundos

Time taken by reduction operation using a function with numba: 49.3 ms ± 359 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)
Time taken by reduction operation using a function with numba and parallel true: 11.7 ms ± 41 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)

En los resultados obtenidos podemos ver un análisis detallado sobre diferentes estrategias para realizar una operación de reducción (suma) en un array de gran tamaño, concretamente, 5*10^7.

En primer lugar, al usar una función con un bucle for para realizar la suma, el tiempo de ejecución es considerablemente mayor. Esto se debe principalmente a que el código Python interpretado carece de optimizaciones avanzadas sin aprovechar las capacidades del hardware ni de las bibliotecas especializadas. Por otro lado, las operaciones realizadas con NumPy, específicamente usando numpy.sum() y numpy.ndarray.sum(), son mucho más rápidas. Este resultado se explica por la implementación subyacente de NumPy en lenguajes de bajo nivel como C, que permite aprovechar eficientemente las arquitecturas modernas.

En segundo lugar, cuando se utiliza paralelismo mediante procesos, el rendimiento mejora apreciablemente a medida que aumenta el número de procesos. Con un solo proceso, sorprendentemente, el tiempo de ejecución (4.84 segundos) es peor que el de la versión secuencial, debido al overhead asociado con la gestión de procesos. Sin embargo, al usar dos procesos, el tiempo se reduce casi a la mitad, y con cuatro procesos alcanza un tiempo aún menor. Esto demuestra que dividir la carga de trabajo entre múltiples núcleos puede ser altamente efectivo para operaciones de reducción intensivas, aunque el manejo del paralelismo introduce cierta complejidad adicional.

En tercer lugar, usando Numba, el tiempo de ejecución sin paralelismo es menor que el observado en la primer celda, lo cual ya es una mejora notable sobre la función personalizada. No obstante, activando la paralelización (parallel=True), el tiempo aún se reduce más, hasta los 11.7 milisegundos. Este es el menor tiempo observado, por tanto, usando numba y la paralelización obtendriamos los mejores resultados en este caso.  
